In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install requests_html

  Obtaining dependency information for fake-useragent from https://files.pythonhosted.org/packages/33/c9/ff44922639b8827dbc86d463d870dabfc19d1567d8a6427dcb2289d83fd8/fake_useragent-1.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for parse from https://files.pythonhosted.org/packages/27/da/2c9c45c3cca6ef6ebf8b4974bf211b523ad8064a315448372f3088a2193e/parse-1.20.0-py2.py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for w3lib from https://files.pythonhosted.org/packages/82/e2/dcf8573d7153194eb673347cea1f9bbdb2a8e61030740fb6f50e4234a00b/w3lib-2.1.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 7.8 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=8238b3689590a88270e11c8f177c145b098691b256e247422c85406d0fb83779
  Stored in directory: /root/.cache/pip/wheels/

In [2]:
!pip install langchain
# !pip install -q newspaper3k python-dotenv
!pip install sentence_transformers===2.2.2
!pip install selenium
!pip install requests_html
# !pip install "git+https://github.com/PanQiWei/AutoGPTQ.git@v0.4.2"
!pip install accelerate
!pip install -U huggingface_hub
!pip install -U hf-transfer
!pip install faiss-gpu
!pip install rank_bm25
!pip install dataclasses-json==0.6.1
!pip install langchain streamlit
!pip install wikipedia

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/ec/f3/b8968f2711b2da26788bd24a19d28a7a21b7a22c4b0f136407ba1526095b/langchain-0.0.354-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.1,>=0.0.8 from https://files.pythonhosted.org/packages/2f/91/0f9ce51f60f5e35fff85a094f0254f22e1645bd4fcfb69fe0cab1b1bc5ea/langchain_community-0.0.8-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2,>=0.1.5 from https://files.pythonhosted.org/packages/5e/2f/6c6415ad9108effa62cdea189ac7ad31503ffc965f1a9ee56f2ed866f688/langchain_core-0.1.5-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.1.0,>=0.0.77 from https://files.pythonhosted.org/packages/c7/a1/6d5

In [5]:
import os
os.environ['HF_HUB_ENABLE_HF_TRANSFER']='1'
!huggingface-cli download --resume-download THUDM/chatglm3-6b --local-dir /kaggle/working/chatglm3-6b

downloading https://huggingface.co/THUDM/chatglm3-6b/resolve/b098244a71fbe69ce149682d9072a7629f7e908c/.gitattributes to /root/.cache/huggingface/hub/models--THUDM--chatglm3-6b/blobs/a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete
.gitattributes: 100%|██████████████████████| 1.52k/1.52k [00:00<00:00, 10.8MB/s]
downloading https://huggingface.co/THUDM/chatglm3-6b/resolve/b098244a71fbe69ce149682d9072a7629f7e908c/MODEL_LICENSE to /root/.cache/huggingface/hub/models--THUDM--chatglm3-6b/blobs/70ddc809f396cd1d05827a86d159fb72553ff5a9.incomplete
MODEL_LICENSE: 100%|███████████████████████| 4.13k/4.13k [00:00<00:00, 32.0MB/s]
downloading https://huggingface.co/THUDM/chatglm3-6b/resolve/b098244a71fbe69ce149682d9072a7629f7e908c/README.md to /root/.cache/huggingface/hub/models--THUDM--chatglm3-6b/blobs/7d7189ada4429ef969537867a7cff9c902f29d59.incomplete
README.md: 100%|███████████████████████████| 7.38k/7.38k [00:00<00:00, 45.8MB/s]
downloading https://huggingface.co/THUDM/chatglm3-6b/resolve/

In [6]:
from langchain.llms.base import LLM
from typing import Dict, List, Any, Optional
import torch,sys,os
from transformers import AutoTokenizer
from langchain.chains import ConversationalRetrievalChain

class GLM(LLM):
  max_token: int = 2048
  temperature: float = 0.8
  top_p = 0.9
  tokenizer: object = None
  model: object = None
  history_len: int = 1024

  def __init__(self, model_name_or_path, bit4=True):
    super().__init__()
    self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,device_map= 'auto',trust_remote_code=True)
    if bit4==False:
        from transformers import AutoModelForCausalLM,AutoModel
        # self.model = AutoModelForCausalLM.from_pretrained(model_name_or_path,device_map='auto',torch_dtype=torch.float16,load_in_8bit=True,trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name_or_path, device_map= 'auto', trust_remote_code=True).half().cuda()
        self.model.eval()
    else:
        from auto_gptq import AutoGPTQForCausalLM
        self.model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,low_cpu_mem_usage=True, device="cuda:0", use_triton=False,inject_fused_attention=False,inject_fused_mlp=False)

    if torch.__version__ >= "2" and sys.platform != "win32":
        self.model = torch.compile(self.model)


  @property
  def _llm_type(self) -> str:
    return "GLM"

  def _call(self, prompt:str, history:List[str]=[], stop:Optional[List[str]]=None):
    response, _ = self.model.chat(
        self.tokenizer,
        prompt,
        history=history[-self.history_len:] if self.history_len > 0 else [],
        max_length=self.max_token,
        temperature=self.temperature,
        top_p=self.top_p)
    return response

llm = GLM(model_name_or_path='/kaggle/working/chatglm3-6b', bit4=False)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "thenlper/gte-base-zh"

model_kwargs = {'device':'cuda'}

hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/32.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/917 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

In [10]:
import pandas as pd
docs = pd.read_csv('/kaggle/input/songss/songs.csv')
docs.head()

,Unnamed: 0,urls,names,texts
0,0,https://music.163.com/song?id=2112831378,慢冷·2024,"作词 : 姚若龙,作曲 : 萧煌奇,编曲 : 骆亚新/于山川,制作人 : 于山川,原唱 : ..."
1,1,https://music.163.com/song?id=190233,偷心,"作词 : 丁晓雯,作曲 : 郭子,是寂寞慢慢占领我的心,长夜里愈来愈冷清,回忆里愈来愈孤寂,..."
2,2,https://music.163.com/song?id=108493,我还想她 - (I Still Miss Her),"作词 : 邢增华,作曲 : 林俊杰,编曲 : 吴庆隆,制作人 : 许环良,我还想她,作词：邢..."
3,3,https://music.163.com/song?id=2078006649,Isle of Skye（空岛）,"作词 : 张寻,作曲 : 张寻,编曲 : 王客观,Over the hill越过山丘,In ..."
4,4,https://music.163.com/song?id=108134,学不会 - (Lost N Found),"作词 : 姚若龙,作曲 : 林俊杰,编曲 : Terence Teo,制作人 : 李偲菘,你..."


In [20]:
from langchain.prompts import PromptTemplate,FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from tqdm import tqdm

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

template = """
我正在构建一个歌曲检索系统。对以下歌词

{song}

您的任务是列出 8 种情绪的列表，稍后我将用它来检索歌曲。

请提供以逗号分隔的情绪列表
"""
prompt = PromptTemplate(
    input_variables=["song"],
    template=template,
    partial_variables={"format_instructions": format_instructions},
)

chain = LLMChain(llm=llm,prompt=prompt,output_parser=output_parser)

documents = []

for doc in tqdm(docs.itertuples(index=False)):
    response = chain.run(doc[3])[0]
    documents.append(Document(page_content=response, metadata={'url':doc[1],'name':doc[2],'text':doc[3]}))

91it [05:07,  3.27s/it]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 3344, but `max_length` is set to 2048. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
100it [05:38,  3.39s/it]


In [23]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents, hf)
db.save_local("/kaggle/working/songs_index")
retriever  = db.as_retriever(search_kwargs={"k": 5})

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
from langchain.prompts import PromptTemplate,FewShotPromptTemplate
examples = [
    {
        "input": "我度过了愉快的一天！",
        "answer": "喜悦"
    }, {
        "input": "我今天很累，感觉不舒服。",
        "answer": "疲惫、不适、疲劳"
    },{
        "input": "我恋爱了。",
        "answer": "开心、爱"
    }
]
example_template="""
User: {input}
AI: {answer}
"""
example_prompt = PromptTemplate(input_variables=['input','answer'], template=example_template)

prefix = """我们有一个简单的歌曲检索系统。它接受八种情绪。您的任务是建议 1 到 4 种情绪来匹配用户的感受。
对于较长的句子建议更多的情感，对于小句子建议一两种情感，试图浓缩输入的中心主题。以下是您过去执行此操作的一些示例：
"""

suffix = """
User: {input}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['input'],
    example_separator="\n"
)
chain = LLMChain(llm=llm,prompt=few_shot_prompt_template,output_parser=output_parser)

In [62]:
response = chain.run('今天中了大奖')
print(response[0])

喜悦、惊喜、幸福、兴奋


In [65]:
searchs = db.similarity_search_with_score(response[0], distance_metric="cos", k=5)
for i in searchs:
    print(f"歌曲名字：{i[0].metadata['name']}")
    print(f"歌曲情感：{i[0].page_content}")
    print(f"链接：{i[0].metadata['url']}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

歌曲名字：リセット - (重置)
歌曲情感：1. 激动
2. 自信
3. 悲伤
4. 快乐
5. 愤怒
6. 惊讶
7. 害怕
8. 满足
链接：https://music.163.com/song?id=1311347602
歌曲名字：Isle of Skye（空岛）
歌曲情感：1. 兴奋
2. 悲伤
3. 愉快
4. 愤怒
5. 惊讶
6. 恐惧
7. 希望
8. 失望
链接：https://music.163.com/song?id=2078006649
歌曲名字：You & Me
歌曲情感：1. 兴奋
2. 心动
3. 希望
4. 坚定
5. 欣赏
6. 满足
7. 期待
8. 爱
链接：https://music.163.com/song?id=2088473570
歌曲名字：如果可以·2023
歌曲情感：1. 悲伤
2. 欢乐
3. 怀念
4. 紧张
5. 惊讶
6. 兴奋
7. 沉思
8. 期待
链接：https://music.163.com/song?id=2029895485
歌曲名字：爱情转移 - (电影《爱情呼叫转移》主题曲)
歌曲情感：1. 悲伤
2. 幸福
3. 怀念
4. 焦虑
5. 兴奋
6. 平静
7. 失望
8. 希望
链接：https://music.163.com/song?id=65536
